In [8]:
import os
import numpy as np
import itertools
import time
from scipy.spatial import distance
from scipy.optimize import fsolve
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [9]:
class PolynomialRegression(object):
    def __init__(self, degree=3, coeffs=None):
        self.degree = degree
        self.coeffs = coeffs

    def fit(self, X, y):
        self.coeffs = np.polyfit(X.ravel(), y, self.degree)

    def get_params(self, deep=False):
        return {'coeffs': self.coeffs}

    def set_params(self, coeffs=None, random_state=None):
        self.coeffs = coeffs

    def predict(self, X):
        poly_eqn = np.poly1d(self.coeffs)
        y_hat = poly_eqn(X.ravel())
        return y_hat

    def score(self, X, y):
        return mean_squared_error(y, self.predict(X))

In [10]:
grid_dict = {}
for y in (-1,1):
    grid_list = [] 

    for x in range(5, -5, -1):
        if y == -1:
            grid_x_up_bound = x*8
            grid_x_low_bound = (x-1)*8
            grid_y_up_bound = (y+1)*3.9
            grid_y_low_bound = y*3.9
            grid = [grid_x_up_bound, grid_x_low_bound, grid_y_up_bound, grid_y_low_bound]
            coord = (y, x)
            grid_dict[coord] = grid
        else:
            grid_x_up_bound = x*8
            grid_x_low_bound = (x-1)*8
            grid_y_up_bound = y*3.9
            grid_y_low_bound = (y-1)*3.9
            grid = [grid_x_up_bound, grid_x_low_bound, grid_y_up_bound, grid_y_low_bound]
            coord = (y, x)
            grid_dict[coord] = grid
        

In [11]:
data_folder = '/Users/jeffeehsiung/Desktop/master thesis/Seoul_Robotics_lane_detection/processed_lidar'
lidar_files = sorted(os.listdir(data_folder))
lidar_fpath = [os.path.join(data_folder, f) for f in lidar_files]
bin = 2
lidar_bin = lidar_fpath[bin]
lidar_data = np.fromfile(lidar_bin, dtype=np.float32).reshape(-1, 3)

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [12]:
lidar_data[:, 2] = 0
data_in_grid = {}
for grid_cell_index, grid_cell_coord in enumerate(grid_dict):
    x_upper_bound, x_lower_bound, y_upper_bound, y_lower_bound = grid_dict[grid_cell_coord]
    inGrid_lidar_data = np.delete(lidar_data, np.where(
    (lidar_data[:, 0] > x_upper_bound) |
    (lidar_data[:, 0] <= x_lower_bound) |
    (lidar_data[:, 1] > y_upper_bound) |
    (lidar_data[:, 1] <= y_lower_bound))[0], axis = 0)
    
    data_in_grid[grid_cell_coord] = inGrid_lidar_data

In [13]:
def cost(up_lane_coeffs, down_lane_coeffs):
    
    x_list = list(range(-40, 41, 1))
    interval_truth = list(itertools.repeat(3, 81)) 
    interval_measured = [] 

    for x_test in x_list:
        y_down = down_lane_coeffs[0]*x_test**3 + down_lane_coeffs[1]*x_test**2 + down_lane_coeffs[2]*x_test + down_lane_coeffs[3]
        deriv_y_down = 3*down_lane_coeffs[0]*x_test**2 + 2*down_lane_coeffs[1]*x_test+down_lane_coeffs[2]

        y_down_perp_line = lambda x: -1/deriv_y_down*(x-x_test) + y_down
        y_up = lambda x: up_lane_coeffs[0]*x**3 + up_lane_coeffs[1]*x**2 + up_lane_coeffs[2]*x + up_lane_coeffs[3]

        intersection_x = findIntersection(y_down_perp_line, y_up, x_test)[0]
        intersection_y = y_down_perp_line(intersection_x)

        dist = distance.euclidean([intersection_x, intersection_y], [x_test, y_down])
        interval_measured.append(dist)
    
    cost = mean_squared_error(interval_truth, interval_measured)
    return cost 

def findIntersection(fun1, fun2, x0):
    return fsolve(lambda x: fun1(x) - fun2(x), x0)

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [14]:
min_sample_threshold = 5
data_repres_up = np.empty((0, 3)) 
data_repres_down = np.empty((0, 3)) 
iteration = 0
max_iter = 1000
prev_error = 1000

start = time.time()
while iteration <= max_iter:
    for y in (-1, 1):
        for x in range(5, -5, -1):
            if y == -1:
                if len(data_in_grid[y, x]) >= min_sample_threshold:
                    idx = np.random.randint(len(data_in_grid[y, x]), size = 3)
                    data_repres_up = np.append(data_repres_up, data_in_grid[y, x][idx, :], axis = 0)
                elif len(data_in_grid[y, x]) == 0:
                    pass
                else:
                    idx = np.random.randint(len(data_in_grid[y, x]), size = len(data_in_grid[y,x]))
                    data_repres_up = np.append(data_repres_up, data_in_grid[y, x][idx, :], axis = 0)

            elif y == 1:
                if len(data_in_grid[y, x]) >= min_sample_threshold:
                    idx = np.random.randint(len(data_in_grid[y, x]), size = 3)
                    data_repres_down = np.append(data_repres_down, data_in_grid[y, x][idx, :], axis = 0)
                elif len(data_in_grid[y, x]) == 0:
                    pass
                else:
                    idx = np.random.randint(len(data_in_grid[y, x]), size = len(data_in_grid[y,x]))
                    data_repres_up = np.append(data_repres_up, data_in_grid[y, x][idx, :], axis = 0)
                    
    up_grids_x = np.sort(data_repres_up, axis = 0)[:, 0] 
    up_grids_y = np.sort(data_repres_up, axis = 0)[:, 1]
    down_grids_x = np.sort(data_repres_down, axis = 0)[:, 0] 
    down_grids_y = np.sort(data_repres_down, axis = 0)[:, 1]

    ransac_up = RANSACRegressor(PolynomialRegression(degree=3),
                             min_samples = 7,    
                             max_trials = 10000,
                             random_state=0)
    ransac_up.fit(np.expand_dims(up_grids_x, axis=1), up_grids_y)
    up_grids_y_pred = ransac_up.predict(np.expand_dims(up_grids_x, axis=1))
    up_lane_coeffs= ransac_up.estimator_.get_params(deep = True)["coeffs"]

    ransac_down = RANSACRegressor(PolynomialRegression(degree=3), 
                             min_samples = 7,    
                             max_trials = 10000,
                             random_state=0)
    ransac_down.fit(np.expand_dims(down_grids_x, axis=1), down_grids_y)
    down_grids_y_pred = ransac_down.predict(np.expand_dims(down_grids_x, axis=1))
    down_lane_coeffs = ransac_down.estimator_.get_params(deep = True)["coeffs"]
    
    ego_lane_coeffs_pair = np.append(down_lane_coeffs, up_lane_coeffs, axis = 0)       
    curr_error = cost(up_lane_coeffs, down_lane_coeffs)
    
    if curr_error < prev_error:
        prev_error = curr_error 
        best_coeffs_pair = ego_lane_coeffs_pair

    iteration += 1
    end = time.time()
    print(end - start)
    print(iteration)

print("time spent: {:.2f} sec".format(time.time() - start)) 
print(best_coeffs_pair)
print(prev_error)

# saved as txt file in the sample_output folder in the format of each line of four coefficients per time 
# ... rest of your code ...

# Convert the best_coeffs_pair to a 2D array with 4 columns
best_coeffs_pair = best_coeffs_pair.reshape(-1, 4)

# Save the coefficients to a text file, with each number formatted as a floating-point number
# get the name of file in pointclouds folder with extension .bin
# and replace the extension with .txt
data_folder = '/Users/jeffeehsiung/Desktop/master thesis/Seoul_Robotics_lane_detection/pointclouds'
lidar_files = sorted(os.listdir(data_folder))
lidar_fpath = [os.path.join(data_folder, f) for f in lidar_files]
lidar_bin = lidar_fpath[bin]
lidar_bin_name = os.path.basename(lidar_bin)
lidar_txt_name = lidar_bin_name.replace('.bin', '.txt')
# save the file in the sample_output folder
# and numbers separated by semicolons
np.savetxt(os.path.join('sample_output', lidar_txt_name), best_coeffs_pair, fmt='%.15e', delimiter=';')

# ... rest of your code ...


0.023682832717895508
1
0.045217037200927734
2
0.06565213203430176
3
0.08708310127258301
4
0.10775995254516602
5
0.12791204452514648
6
0.15064597129821777
7
0.1714489459991455
8
0.21898603439331055
9
0.24122405052185059
10
0.2575209140777588
11
0.2799060344696045
12
0.30538201332092285
13
0.34240221977233887
14
0.36602091789245605
15
0.39184117317199707
16
0.41909193992614746
17
0.45061802864074707
18
0.4949038028717041
19
0.5276198387145996
20
0.5517079830169678
21
0.5856719017028809
22
0.619163990020752
23
0.6468181610107422
24
0.6702852249145508
25
0.6900460720062256
26
0.7126522064208984
27
0.7432188987731934
28
0.7613620758056641
29
0.7865140438079834
30
0.8109698295593262
31
0.8347609043121338
32
0.8567581176757812
33
0.8821568489074707
34
0.9270038604736328
35
0.9632089138031006
36
1.006059169769287
37
1.0452938079833984
38
1.1604628562927246
39
1.2205309867858887
40
1.2982230186462402
41
1.4213190078735352
42
1.4814729690551758
43
1.5080249309539795
44
1.5318830013275146
45
1.55

2.385672092437744
72
2.442434787750244
73
2.4739530086517334
74
2.4914510250091553
75
2.5190460681915283
76
2.543588876724243
77
2.5648021697998047
78
2.5901169776916504
79
2.6140739917755127
80
2.636559009552002
81
2.657217025756836
82
2.685436964035034
83
2.7114689350128174
84
2.73740816116333
85
2.7603888511657715
86
2.8009140491485596
87
2.939162015914917
88
3.0090320110321045
89
3.041975975036621
90
3.096306085586548
91
3.1318678855895996
92
3.1769540309906006
93
3.215254068374634
94
3.2603960037231445
95
3.3039748668670654
96
3.349705934524536
97
3.392082929611206
98
3.4364209175109863
99
3.4718830585479736
100
3.5102379322052
101
3.5456631183624268
102
3.641211986541748
103
3.6904380321502686
104
3.720982074737549
105
3.7458791732788086
106
3.7717461585998535
107
3.8004350662231445
108
3.8358888626098633
109
3.866663932800293
110
3.9144949913024902
111
3.951137065887451
112
3.991802930831909
113
4.0298309326171875
114
4.06361198425293
115
4.115793943405151
116
4.155511856079102


ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

# cost function
